# Exercise

7. Train and fine-tune a decision tree for the moons dataset by following these steps:
a. Use make_moons(n_samples=10000, noise=0.4) to generate a moons dataset.
b. Use train_test_split() to split the dataset into a training set and a test set.
c. Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes.
d. Train it on the full training set using these hyperparameters, and measure your model’s performance on the test set. You should get roughly 85% to 87% accuracy.


8. Grow a forest by following these steps:
a. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit- Learn’s ShuffleSplit class for this.
b. Train one decision tree on each subset, using the best hyperparameter values found in the previous exercise. Evaluate these 1,000 decision trees on the test set. Since they were trained on smaller sets, these decision trees will likely perform worse than the first decision tree, achieving only about 80% accuracy.
c. Now comes the magic. For each test set instance, generate the predictions of the 1,000 decision trees, and keep only the most frequent prediction (you can use SciPy’s mode() function for this). This approach gives you majority-vote predictions over the test set.
d. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a random forest classifier!

In [24]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split,cross_val_predict,cross_val_score,GridSearchCV
import pandas as pd
from sklearn.pipeline import Pipeline
import numpy as np
# from scipy.stats import ('standard',StandardScaler()),('pca',PCA()),


X,y = make_moons(n_samples=10000, noise=0.4,random_state=42)
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.2)



In [25]:
from sklearn.tree import DecisionTreeClassifier
tree_clf =DecisionTreeClassifier(random_state=42)
# tree_clf.fit(X_train,y_train)
tree_pipeline = Pipeline([('tree_clf',DecisionTreeClassifier(random_state=42))])
params_grid = {'tree_clf__max_leaf_nodes':[2,4,6,8,10],
              'tree_clf__max_depth':[1,2,4,6,8,10],
              'tree_clf__max_features':[1,2,3],
              'tree_clf__min_samples_split':[1,2,4,6,8,10]}

params = {
    'tree_clf__max_leaf_nodes': list(range(2, 100)),
    'tree_clf__max_depth': list(range(1, 7)),
    'tree_clf__min_samples_split': [2, 3, 4]}
    
tree_grid = GridSearchCV(tree_pipeline,param_grid=params,cv=5,scoring='accuracy')
tree_grid.fit(X_train,y_train)





GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tree_clf',
                                        DecisionTreeClassifier(random_state=42))]),
             param_grid={'tree_clf__max_depth': [1, 2, 3, 4, 5, 6],
                         'tree_clf__max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, 30, 31, ...],
                         'tree_clf__min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [47]:
tree_grid.best_params_

{'tree_clf__max_depth': 6,
 'tree_clf__max_leaf_nodes': 22,
 'tree_clf__min_samples_split': 2}

In [27]:
tree_grid.best_score_

0.8564999999999999

In [28]:
tree_grid.score(X_test,y_test)

0.8595

In [29]:
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score
from scipy.stats import mode

In [31]:
shuffle_split = ShuffleSplit(n_splits=1000,train_size=100,random_state=42)
# train_indices,test_indices = next(shuffle_split.split(X_train,y_train))
splits = shuffle_split.split(X_train,y_train)

In [32]:
pred =[]
for train_indices, test_indices in splits:
    # Accessing the data with the indices
    X_train1, y_train1 = X[train_indices], y[train_indices]
    dec_tree = DecisionTreeClassifier(max_depth=6,max_leaf_nodes=22,min_samples_split=2)
    dec_tree.fit(X_train1,y_train1)
#     pred.append(accuracy_score(y_test,dec_tree.predict(X_test)))
    test_pred =dec_tree.predict(X_test)
    pred.append(test_pred)
array_2d = np.vstack(pred)

In [33]:
accuracy_score(mode(pred,axis=0)[0][0],y_test)


0.8755